In [1]:
import CNN_lib
import CNN_utilities
import fig_lib 
import CNN_machine

import numpy as np
import pickle

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# GENERAL PARAMETERS

In [2]:

checkpoint_path = "C:/PROJECTS\emboendo\CNN\python-machine-learning\emboendo_ML\embo_CNN_lab\_static/checkpoints"
pkl_train_p='C:\PROJECTS\emboendo\CNN/pikles/processed_train_d.pkl'  

number_data_ls= [i for i in range(48, 530, 32)]
check_points_p = [checkpoint_path+'_'+str(n)+'.h5' for n in number_data_ls]
models_perf_d={'model'+'_'+str(n):{} for n in number_data_ls}

test_size=0.2
epochs=7

print(len(number_data_ls),number_data_ls)
print(models_perf_d)


16 [48, 80, 112, 144, 176, 208, 240, 272, 304, 336, 368, 400, 432, 464, 496, 528]
{'model_48': {}, 'model_80': {}, 'model_112': {}, 'model_144': {}, 'model_176': {}, 'model_208': {}, 'model_240': {}, 'model_272': {}, 'model_304': {}, 'model_336': {}, 'model_368': {}, 'model_400': {}, 'model_432': {}, 'model_464': {}, 'model_496': {}, 'model_528': {}}


## DATA LOAD

In [3]:
with open(pkl_train_p, 'rb') as pikle_file:
    save_trains= pickle.load(pikle_file)

Xx_train,Yy_train,rcs =  save_trains[0],save_trains[1],save_trains[2]
max_frm_n, HEIGHT, WIDTH=np.shape(Xx_train)[1],np.shape(Xx_train)[2],np.shape(Xx_train)[3]
input_shape=(max_frm_n, HEIGHT, WIDTH, 1)

## MAIN DATA SHUFFLE

In [4]:
Xx_train,Yy_train,recs=CNN_lib.shuffle(Xx_train,Yy_train,rcs)

## DATA SPLIT

In [5]:
test_size=0.2
X_train_spl, X_eval_spl, Y_train_spl, Y_eval_spl = train_test_split(Xx_train, Yy_train, test_size=test_size, shuffle=False) #,random_state=42)

recs_train =recs[0:len(Y_train_spl)]
recs_eval =recs[len(Y_train_spl)::]

## DATA AUGMENTATION

In [7]:
nf1_ls,nf0_ls,nc1_ls,nc0_ls,nb1_ls,nb0_ls= [],[],[],[],[],[]

for nn in number_data_ls:

    zeros_count = np.sum(Y_train_spl[0:nn] == 0)
    ones_count= np.sum(Y_train_spl[0:nn] == 1)

    n_flip_1=1
    n_flip_0=0.07

    nf1_ls.append(round(n_flip_1*ones_count))
    nf0_ls.append(round(n_flip_0*zeros_count))

    n_contr_1=0.4
    n_contr_0=0.03

    nc1_ls.append(round(n_contr_1*ones_count))
    nc0_ls.append(round(n_contr_0*zeros_count))

    n_bright_1=0.4
    n_bright_0=0.04

    nb1_ls.append(round(n_bright_1*ones_count))
    nb0_ls.append(round(n_bright_0*zeros_count))


Samples: 48 ,Flip 0: 2 ,Flip 1: 15 ,Contr 0: 1 ,Contr 1: 6 ,Bright 0: 1 ,Bright 1: 6
Samples: 80 ,Flip 0: 4 ,Flip 1: 28 ,Contr 0: 2 ,Contr 1: 11 ,Bright 0: 2 ,Bright 1: 11
Samples: 112 ,Flip 0: 6 ,Flip 1: 32 ,Contr 0: 2 ,Contr 1: 13 ,Bright 0: 3 ,Bright 1: 13
Samples: 144 ,Flip 0: 7 ,Flip 1: 37 ,Contr 0: 3 ,Contr 1: 15 ,Bright 0: 4 ,Bright 1: 15
Samples: 176 ,Flip 0: 9 ,Flip 1: 46 ,Contr 0: 4 ,Contr 1: 18 ,Bright 0: 5 ,Bright 1: 18
Samples: 208 ,Flip 0: 11 ,Flip 1: 53 ,Contr 0: 5 ,Contr 1: 21 ,Bright 0: 6 ,Bright 1: 21
Samples: 240 ,Flip 0: 13 ,Flip 1: 61 ,Contr 0: 5 ,Contr 1: 24 ,Bright 0: 7 ,Bright 1: 24
Samples: 272 ,Flip 0: 14 ,Flip 1: 68 ,Contr 0: 6 ,Contr 1: 27 ,Bright 0: 8 ,Bright 1: 27
Samples: 304 ,Flip 0: 16 ,Flip 1: 78 ,Contr 0: 7 ,Contr 1: 31 ,Bright 0: 9 ,Bright 1: 31
Samples: 336 ,Flip 0: 17 ,Flip 1: 89 ,Contr 0: 7 ,Contr 1: 36 ,Bright 0: 10 ,Bright 1: 36
Samples: 368 ,Flip 0: 19 ,Flip 1: 101 ,Contr 0: 8 ,Contr 1: 40 ,Bright 0: 11 ,Bright 1: 40
Samples: 400 ,Flip 0: 20 ,F

In [ ]:
X_train , Y_train = [] , []

for nn,nf1,nf0,nc1,nc0,nb1,nb0 in zip(number_data_ls,nf1_ls,nf0_ls,nc1_ls,nc0_ls,nb1_ls,nb0_ls):

    print('Samples:',nn,',Flip 0:',nf0, ',Flip 1:',nf1,',Contr 0:',nc0,',Contr 1:',nc1,',Bright 0:',nb0,',Bright 1:',nb1)

    
    Flip_X_1,Flip_Y_1,Flip_recs_1=CNN_lib.main_aug_f(nf1,X_train_spl[0:nn],Y_train_spl[0:nn],recs[0:nn],label=1,typ='Flip')
    Flip_X_0,Flip_Y_0,Flip_recs_0=CNN_lib.main_aug_f(nf0,X_train_spl[0:nn],Y_train_spl[0:nn],recs[0:nn],label=0,typ='Flip')

    Cntr_X_1,Cntr_Y_1,Cntr_recs_1=CNN_lib.main_aug_f(nc1,X_train_spl[0:nn],Y_train_spl[0:nn],recs[0:nn],label=1,typ='Contrast')
    Cntr_X_0,Cntr_Y_0,Cntr_recs_0=CNN_lib.main_aug_f(nc0,X_train_spl[0:nn],Y_train_spl[0:nn],recs[0:nn],label=0,typ='Contrast')

    Bgr_X_1,Bgr_Y_1,Bgr_recs_1=CNN_lib.main_aug_f(nb1,X_train_spl[0:nn],Y_train_spl[0:nn],recs[0:nn],label=1,typ='Brightness')
    Bgr_X_0,Bgr_Y_0,Bgr_recs_0=CNN_lib.main_aug_f(nb0,X_train_spl[0:nn],Y_train_spl[0:nn],recs[0:nn],label=0,typ='Brightness')

    X_train_n = np.concatenate((X_train_spl[0:nn], Flip_X_1,Flip_X_0,Cntr_X_1,Cntr_X_0,Bgr_X_1,Bgr_X_0), axis=0)
    Y_train_n = np.concatenate((Y_train_spl[0:nn], Flip_Y_1, Flip_Y_0,Cntr_Y_1,Cntr_Y_0,Bgr_Y_1,Bgr_Y_0))
    
    recs_n = recs_train[0:nn]+Flip_recs_1+Flip_recs_0+Cntr_recs_1+Cntr_recs_0+Bgr_recs_1+Bgr_recs_0

    X_train.append(X_train_n)
    Y_train.append(Y_train_n)

In [ ]:

for xx, yy,nn in zip(X_train , Y_train,number_data_ls): print('Data:',nn,'X train shape:',xx.shape, 'Y train shape:',yy.shape )
    

# CNN

In [ ]:
models=[CNN_machine.Gen_Model(input_shape,pp) for pp in check_points_p]

In [ ]:
ii=1
print(' ')
for mdl,X,Y in zip(models,X_train,Y_train):
    
    print(f'-----------------{ii}-----------------')
    mdl.train_model(X,Y,X_eval_spl,Y_eval_spl, epochs)
    print('-----------------------------------')
    print(' ')
    ii+=1

In [ ]:
#for mdl in models:CNN_lib.plot_train_eval(mdl.history,epochs)

In [ ]:
for mdl in models: CNN_lib.plot_roc_curve(mdl.fpr_val,mdl.tpr_val,mdl.roc_auc)

In [ ]:
AUC_ls=[mdl.roc_auc for mdl in models]
number_points=[mdl.tr_pnt for mdl in models]

plt.plot(number_points, AUC_ls, marker='o', linestyle='-', color='b', label='AUC')
plt.xlabel('SAMPLES')
plt.ylabel('AUC')
plt.title('AUC VS SAMPLES')
plt.show()